In [2]:
import cv2
import numpy as np
import os
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.model_selection import train_test_split
import copy
from PIL import Image
from collections import defaultdict
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader,Dataset

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_mcs_mouth.xml')

In [19]:
def extract_face(img):
    if not isinstance(img,np.ndarray):
        img=cv2.imread(img) #si on passe un path ça read l'img

    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=6,
        minSize=(60,60)
    )

    realFaces=[]
    if len(faces)==0:
        return []
    else:
        for k,face in enumerate(faces):
            x,y,w,h=face
            eyes=eye_cascade.detectMultiScale(gray[y:y+h,x:x+w])
            #mouth=mouth_cascade.detectMultiScale(gray[y:y+h,x:x+w])

            if len(eyes)>=2:
                realFaces.append(face)

    return realFaces

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [29]:
capture=cv2.VideoCapture(0,cv2.CAP_DSHOW)
path="TestPhotos"
u=0

modelPath="resNet50V1.pth"
trainingPath="CameraFaces"
trainingFile=os.listdir(trainingPath)
device = torch.device("cpu")
model = models.resnet50(pretrained=False)
#nb_classes=len(trainingFile)
nb_classes=2
model.fc=nn.Linear(model.fc.in_features,nb_classes)
model.load_state_dict(torch.load(modelPath,map_location=device))
model=model.to(device)
model.eval()

if not capture.isOpened():
    print("Erreur : Impossible d'ouvrir la webcam")
else:
    while True:
        ret,frame=capture.read()

        if not ret:
            break

        #frame=cv2.rotate(frame,cv2.ROTATE_90_CLOCKWISE)
        faces=extract_face(frame)
        if len(faces)!=0:
            u+=1
            for i,face in enumerate(faces):
                x,y,w,h=face
                crop=frame[y:y+h,x:x+w]
                img_tensor=transform(Image.fromarray(crop)).unsqueeze(0).to(device)
                with torch.no_grad():
                        output=model(img_tensor)
                        pred=torch.argmax(output,dim=1)
                name=trainingFile[pred]
                cv2.rectangle(frame,(x-2,y-2),(x+w+2,y+h+2),(0,255,255),2)
                cv2.putText(frame,text=name,org=(x+w//4,y+h+30),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=0.75,color=(255, 255, 255),thickness=1,lineType=cv2.LINE_AA)
                if u>=30:
                    u=0
                    file=os.listdir(path)
                    if f"{path}/photo-{len(file)}.jpg" not in file:
                        cv2.imwrite(f"{path}/photo-{len(file)}.jpg",frame[y:y+h,x:x+w])
                        print(f"Image enregistrée : {len(file)}")
                cv2

        cv2.imshow("Webcam",frame)
        if cv2.waitKey(1) & 0xFF==ord("q"):
            break

capture.release()
cv2.destroyAllWindows()
print("Program exited")

c:\Users\asami\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\asami\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Image enregistrée : 6
Image enregistrée : 7
Image enregistrée : 8
Image enregistrée : 9
Image enregistrée : 10
Image enregistrée : 11
Image enregistrée : 12
Program exited
